In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Breast_Cancer"
cohort = "GSE249377"

# Input paths
in_trait_dir = "../../input/GEO/Breast_Cancer"
in_cohort_dir = "../../input/GEO/Breast_Cancer/GSE249377"

# Output paths
out_data_file = "../../output/preprocess/Breast_Cancer/GSE249377.csv"
out_gene_data_file = "../../output/preprocess/Breast_Cancer/gene_data/GSE249377.csv"
out_clinical_data_file = "../../output/preprocess/Breast_Cancer/clinical_data/GSE249377.csv"
json_path = "../../output/preprocess/Breast_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Exploring the Effects of Experimental Parameters and Data Modeling Approaches on In Vitro Transcriptomic Point-of-Departure Estimates"
!Series_summary	"Multiple new approach methods (NAMs) are being developed to rapidly screen large numbers of chemicals to aid in hazard evaluation and risk assessments. High-throughput transcriptomics (HTTr) in human cell lines has been proposed as a first-tier screening approach for determining the types of bioactivity a chemical can cause (activation of specific targets vs. generalized cell stress) and for calculating transcriptional points of departure (tPODs) based on changes in gene expression. In the present study, we examine a range of computational methods to calculate tPODs from HTTr data, using six data sets in which MCF7 cells cultured in two different media formulations were treated with a panel of 44 chemicals for 3 different exposure durations (6, 12, 24 hr)."
!Series_overall_design	"Multiple computat

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From the Series summary and overall design, this dataset appears to contain gene expression data
# from high-throughput transcriptomics (HTTr) experiments in MCF7 cells
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# There is no explicit trait (breast cancer) information in the sample characteristics
# The MCF7 cell line is a breast cancer cell line, but this is a constant across all samples
trait_row = None  # No trait variable is available

# Age is not available in this dataset as it's a cell line study
age_row = None

# No gender information is available as this is a cell line study
gender_row = None

# Define conversion functions for trait data (even though not used in this case)
def convert_trait(value):
    # Not used in this dataset, but required for the function signature
    if value is None or 'NA' in value:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    return 1 if value.lower() == 'yes' else 0

def convert_age(value):
    # Not used in this dataset
    if value is None or 'NA' in value:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    # Not used in this dataset
    if value is None or 'NA' in value:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    return 1 if value.lower() in ['male', 'm'] else 0 if value.lower() in ['female', 'f'] else None

# 3. Save Metadata
# Trait data is not available as trait_row is None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                             is_gene_available=is_gene_available, is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
# Skip this step since trait_row is None (no clinical data is available as determined above)


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker and examine file content
print("Examining matrix file content...")
marker_row = None
sample_lines = []

try:
    with gzip.open(matrix_file, 'rt') as file:
        for i, line in enumerate(file):
            # Store marker position if found
            if "!series_matrix_table_begin" in line:
                marker_row = i
                print(f"Found matrix table marker at line {i}")
            
            # Store lines around the marker for inspection
            if marker_row is not None and i >= marker_row and i < marker_row + 10:
                sample_lines.append(line.strip())
            
            # Also capture some lines from the beginning
            if i < 10:
                print(f"Line {i}: {line.strip()[:100]}...")
            
            # Don't read the entire file
            if i > marker_row + 20 and marker_row is not None:
                break
            if i > 100 and marker_row is None:
                break

    if marker_row is not None:
        print("\nLines immediately after the marker:")
        for i, line in enumerate(sample_lines):
            print(f"Line {marker_row + i}: {line[:100]}...")
        
        # Try a more manual approach to extract the data
        data_lines = []
        gene_ids = []
        with gzip.open(matrix_file, 'rt') as file:
            for i, line in enumerate(file):
                if i <= marker_row:  # Skip until after the marker
                    continue
                if line.startswith('!'):  # Skip any remaining comment lines
                    continue
                if not line.strip():  # Skip empty lines
                    continue
                
                # Found a data line
                if i == marker_row + 1:  # This should be the header line
                    headers = line.strip().split('\t')
                    print(f"Found headers: {headers[:5]}... (total: {len(headers)})")
                else:
                    parts = line.strip().split('\t')
                    if len(parts) > 1:  # Ensure it's a valid data line
                        gene_ids.append(parts[0])
                        data_lines.append(parts)

                # Don't process too many lines for this test
                if len(data_lines) > 100:
                    break
        
        if len(data_lines) > 0:
            print(f"Successfully parsed {len(data_lines)} data lines manually")
            print(f"First few gene IDs: {gene_ids[:10]}")
            
            # Now try the proper extraction
            try:
                # Try using the library function
                gene_data = get_genetic_data(matrix_file)
                
                if len(gene_data) > 0:
                    print(f"Successfully extracted gene data with shape: {gene_data.shape}")
                    print("First 20 gene/probe identifiers:")
                    print(gene_data.index[:20].tolist())
                else:
                    # If the library function fails, try direct pandas method
                    print("Library function returned empty data, trying direct pandas method...")
                    gene_data = pd.read_csv(matrix_file, compression='gzip', 
                                            skiprows=marker_row+1, 
                                            header=0, 
                                            sep='\t', 
                                            on_bad_lines='skip')
                    
                    if len(gene_data) > 0:
                        id_col = gene_data.columns[0]
                        gene_data = gene_data.rename(columns={id_col: 'ID'})
                        gene_data.set_index('ID', inplace=True)
                        print(f"Successfully extracted gene data with shape: {gene_data.shape}")
                        print("First 20 gene/probe identifiers:")
                        print(gene_data.index[:20].tolist())
                    else:
                        print("Still couldn't extract gene data using pandas.")
                        is_gene_available = False
            except Exception as e:
                print(f"Error extracting gene data with standard methods: {e}")
                is_gene_available = False
        else:
            print("No data lines found after the marker")
            is_gene_available = False
    else:
        print("Could not find '!series_matrix_table_begin' marker in the file.")
        is_gene_available = False
    
except Exception as e:
    print(f"Error examining file: {e}")
    is_gene_available = False

if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")
    # Update the validation record since gene data isn't available
    is_trait_available = trait_row is not None
    validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                                 is_gene_available=is_gene_available, is_trait_available=is_trait_available)


SOFT file: ../../input/GEO/Breast_Cancer/GSE249377/GSE249377_family.soft.gz
Matrix file: ../../input/GEO/Breast_Cancer/GSE249377/GSE249377_series_matrix.txt.gz
Examining matrix file content...
Line 0: !Series_title	"Exploring the Effects of Experimental Parameters and Data Modeling Approaches on In V...
Error examining file: unsupported operand type(s) for +: 'NoneType' and 'int'
Gene expression data could not be successfully extracted from this dataset.


### Step 4: Gene Data Extraction

In [5]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
marker_row = None
try:
    with gzip.open(matrix_file, 'rt') as file:
        for i, line in enumerate(file):
            if "!series_matrix_table_begin" in line:
                found_marker = True
                marker_row = i
                print(f"Found the matrix table marker at line {i}")
                break
    
    if not found_marker:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        is_gene_available = False
        
    # If marker was found, try to extract gene data
    if is_gene_available:
        try:
            # Try using the library function
            gene_data = get_genetic_data(matrix_file)
            
            if gene_data.shape[0] == 0:
                print("Warning: Extracted gene data has 0 rows.")
                is_gene_available = False
            else:
                print(f"Gene data shape: {gene_data.shape}")
                # Print the first 20 gene/probe identifiers
                print("First 20 gene/probe identifiers:")
                print(gene_data.index[:20].tolist())
        except Exception as e:
            print(f"Error extracting gene data with get_genetic_data(): {e}")
            is_gene_available = False
    
    # If gene data extraction failed, examine file content to diagnose
    if not is_gene_available:
        print("Examining file content to diagnose the issue:")
        try:
            with gzip.open(matrix_file, 'rt') as file:
                # Print lines around the marker if found
                if marker_row is not None:
                    for i, line in enumerate(file):
                        if i >= marker_row - 2 and i <= marker_row + 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        if i > marker_row + 10:
                            break
                else:
                    # If marker not found, print first 10 lines
                    for i, line in enumerate(file):
                        if i < 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        else:
                            break
        except Exception as e2:
            print(f"Error examining file: {e2}")
        
except Exception as e:
    print(f"Error processing file: {e}")
    is_gene_available = False

# Update validation information if gene data extraction failed
if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")
    # Update the validation record since gene data isn't available
    is_trait_available = False  # We already determined trait data isn't available in step 2
    validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                                 is_gene_available=is_gene_available, is_trait_available=is_trait_available)

SOFT file: ../../input/GEO/Breast_Cancer/GSE249377/GSE249377_family.soft.gz
Matrix file: ../../input/GEO/Breast_Cancer/GSE249377/GSE249377_series_matrix.txt.gz
Found the matrix table marker at line 84


Examining file content to diagnose the issue:
Line 82: !Sample_relation	"SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX22782419"	"SRA: https://www.ncbi.nlm...
Line 83: !Sample_supplementary_file_1	"NONE"	"NONE"	"NONE"	"NONE"	"NONE"	"NONE"	"NONE"	"NONE"	"NONE"	"NONE"	"...
Line 84: !series_matrix_table_begin...
Line 85: "ID_REF"	"GSM7937728"	"GSM7937729"	"GSM7937730"	"GSM7937731"	"GSM7937732"	"GSM7937733"	"GSM7937734"	...
Line 86: !series_matrix_table_end...
Gene expression data could not be successfully extracted from this dataset.
